In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cnsenti import Sentiment  # 情感分析

In [2]:
review_path = 'E:/泰迪杯/数据挖掘2021/LDA结果/LDA分主题结果/酒店（含名词）utf-8.csv'
fig_path1 = 'E:/泰迪杯/数据挖掘2021/论文图片/情感分析/离群值/H'
fig_path2 = 'E:/泰迪杯/数据挖掘2021/论文图片/情感分析/评分直方图/H'
result_path_u = 'E:/泰迪杯/数据挖掘2021/情感分析结果utf-8/H'
result_path_g = 'E:/泰迪杯/数据挖掘2021/情感分析结果gbk/H'

In [3]:
reviews = pd.read_csv(review_path, encoding='utf-8')

In [4]:
reviews.head(3)

,酒店名称,评论日期,评论内容,入住房型,主题,评论对主题贡献度,主题关键字,评论分词,评价方面
0,H01,2020-01-01,酒店很适合家庭出行,标准客房,2.0,0.7856,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","['适合', '家庭', '出行']",位置
1,H01,2020-01-01,升级了房间 延迟退房 很赞,标准客房,0.0,0.8565,"服务, 房间, 升级, 入住, 免费, 早餐, 热情, 推荐, 满意, 不错","['升级', '房间', '延迟', '退房', '赞']",服务
2,H01,2020-01-01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房,2.0,0.5995,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","['几年', '广州', '每次', '住', '位置', '性价比', '不错', '免费...",位置


In [5]:
reviews.shape

(20989, 9)

# 数据预处理

## 将评论分词的数据类型从str转为分词的列表

In [6]:
# 输入str
def str2list(x):
    l = x.replace('[\'','').replace('\']','').replace('\'','').replace(' ','')
    l = l.split(',')
    return l 

In [7]:
reviews['评论分词'] = reviews['评论分词'].apply(lambda x: str2list(x))
reviews['酒店编号'] = reviews['酒店名称'].apply(lambda x: int(x[1:3]))

In [8]:
reviews.head()

,酒店名称,评论日期,评论内容,入住房型,主题,评论对主题贡献度,主题关键字,评论分词,评价方面,酒店编号
0,H01,2020-01-01,酒店很适合家庭出行,标准客房,2.0,0.7856,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","[适合, 家庭, 出行]",位置,1
1,H01,2020-01-01,升级了房间 延迟退房 很赞,标准客房,0.0,0.8565,"服务, 房间, 升级, 入住, 免费, 早餐, 热情, 推荐, 满意, 不错","[升级, 房间, 延迟, 退房, 赞]",服务,1
2,H01,2020-01-01,这几年，每年都会来广州，每次都会住**酒店。因为位置好，酒店的性价比也不错，这次给免费升级了...,标准客房,2.0,0.5995,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","[几年, 广州, 每次, 住, 位置, 性价比, 不错, 免费, 升级, 房间]",位置,1
3,H01,2020-01-01,超五星好评,高级客房,4.0,0.5070,"房, 旅游, 服務, 时, 房间, 好评, 游玩, 套房, 选择, 订","[超, 五星, 好评]",设施,1
4,H01,2020-01-01,酒店干净卫生，服务很好！,标准客房,1.0,0.7856,"服务, 干净, 卫生, 房间, 特别, 热情, 温泉, 整洁, 喜欢, 入住","[干净, 卫生, 服务]",卫生,1


## 编制情感分析的数据表

In [9]:
emotion = pd.DataFrame()
emotion['index_server'] = reviews['酒店编号']
emotion['index_content'] = reviews.index.values + 1
emotion['word_list'] = reviews['评论分词']

In [10]:
emotion

,index_server,index_content,word_list
0,1,1,"[适合, 家庭, 出行]"
1,1,2,"[升级, 房间, 延迟, 退房, 赞]"
2,1,3,"[几年, 广州, 每次, 住, 位置, 性价比, 不错, 免费, 升级, 房间]"
3,1,4,"[超, 五星, 好评]"
4,1,5,"[干净, 卫生, 服务]"
...,...,...,...
20984,50,20985,"[不错, 性价比, 高]"
20985,50,20986,"[老字号, 宾馆, 装修, 设施, 陈旧, 房间, 空调, 卫生间, 里, 墙壁, 剥落, ..."
20986,50,20987,"[还好, 价格, 偏, 高, 高档, 服务, 设施, 偏, 很大, 喜欢, 楼下, 早茶]"
20987,50,20988,"[隔音, 极差, 睡眠, 崩溃]"


In [11]:
# 将评论分词的列表张成dataframe
result = pd.DataFrame({'index_server':emotion.index_server.repeat(emotion.word_list.str.len()),'index_content':emotion.index_content.repeat(emotion.word_list.str.len()),'word':np.concatenate(emotion.word_list.values)})
result.reset_index(drop=True,inplace=True)

In [12]:
result

,index_server,index_content,word
0,1,1,适合
1,1,1,家庭
2,1,1,出行
3,1,2,升级
4,1,2,房间
...,...,...,...
229934,50,20988,极差
229935,50,20988,睡眠
229936,50,20988,崩溃
229937,50,20989,位置


## 构建各词在在对应评论的位置列

In [13]:
list(result.groupby(by=['index_content']))

[(1,    index_server  index_content word
  0             1              1   适合
  1             1              1   家庭
  2             1              1   出行), (2,    index_server  index_content word
  3             1              2   升级
  4             1              2   房间
  5             1              2   延迟
  6             1              2   退房
  7             1              2    赞), (3,
      index_server  index_content word
  8              1              3   几年
  9              1              3   广州
  10             1              3   每次
  11             1              3    住
  12             1              3   位置
  13             1              3  性价比
  14             1              3   不错
  15             1              3   免费
  16             1              3   升级
  17             1              3   房间), (4,
      index_server  index_content word
  18             1              4    超
  19             1              4   五星
  20             1              4   好评), (5,
      inde

In [14]:
# 在原有的reviews_n表中，每条评论所提取的词汇长度如下：
n_word = list(result.groupby(by=['index_content'])['index_content'].count())

In [15]:
# 构建index_word列表；
index_word = [list(np.arange(0, y)) for y in n_word]

In [16]:
index_word

[[0, 1, 2],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 [0, 1],
 [0, 1, 2],
 [0, 1],
 [0],
 [0, 1, 2],
 [0, 1],
 [0, 1],
 [0, 1, 2, 3],
 [0, 1, 2, 3],
 [0, 1, 2],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
 [0, 1, 2],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2, 3],
 [0],
 [0, 1],
 [0, 1, 2],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0, 1, 2],
 [0, 1, 2],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
 

In [17]:
# 展开index_word列表，表示词语在改评论的位置；合并评论id，评论中词的id，词，词性；
index_word = sum(index_word, [])
result['index_word'] = index_word

In [18]:
result.head()

,index_server,index_content,word,index_word
0,1,1,适合,0
1,1,1,家庭,1
2,1,1,出行,2
3,1,2,升级,0
4,1,2,房间,1


# 情感分析

In [19]:
senti = Sentiment()
posneg = result['word'].apply(lambda x: senti.sentiment_calculate(x))

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built successfully.


In [20]:
posneg

0         {'sentences': 1, 'words': 1, 'pos': 1, 'neg': 0}
1         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
2         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
3         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
4         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
                                ...                       
229934    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
229935    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
229936    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 1}
229937    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
229938    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
Name: word, Length: 229939, dtype: object

In [21]:
# 判断每个词的情感极性,1积极，0中立，-1消极
result['emotion'] = posneg
result['pos'] = result['emotion'].apply(lambda x: x.get('pos'))
result['neg'] = result['emotion'].apply(lambda x: -x.get('neg'))

In [22]:
# 修正情感倾向：
# 搜索每个正向情感词语前面的两个词语是否有否定词汇。前面有一个否定词语，weight值应该修正为-1，否定词修正为0，稍后去掉；
# 前面两个否定词语，weight值不变，否定词修正为0，稍后去掉
# 输入一行
# 修正后的权重值用amend_weight表示(注释了)
def mend_weight(x):
    IndexContent = x['index_content']
    IndexWord = x['index_word']
    Pos = x['pos']
    Neg = x['neg']
    if Pos != 1:
        return
    if IndexWord == 0: # 第一个词
        return
    if IndexWord == 1: # 第二个词
        filt = (result['index_content']==IndexContent) &  (result['index_word']== 0)
        tmp = result[filt]
        if tmp.iloc[0,5] == -1: # 前一个词是负面词
            x['pos'] = 0
            x['neg'] = -1
            #x['amend_weight'] = -1
            tmp['neg'] = 0
        else:
            return
    filt1 = (result['index_content']==IndexContent) & (result['index_word']== (IndexWord - 2))
    filt2 = (result['index_content']==IndexContent) & (result['index_word']== (IndexWord - 1))
    tmp1 = result[filt1]
    tmp2 = result[filt2]
    
    k = tmp1.iloc[0,5] + tmp2.iloc[0,5]
    if k == -2: # 前两个词是负面词
        tmp1['neg'] = 0
        tmp2['neg'] = 0
    elif k == -1: # 前一个词是负面词
        x['pos'] = 0
        x['neg'] = -1
        tmp1['neg'] = 0
        tmp2['neg'] = 0
    else:
        return    

In [23]:
result.apply(lambda x: mend_weight(x),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
229934    None
229935    None
229936    None
229937    None
229938    None
Length: 229939, dtype: object

In [24]:
result.head()

,index_server,index_content,word,index_word,emotion,pos,neg
0,1,1,适合,0,"{'sentences': 1, 'words': 1, 'pos': 1, 'neg': 0}",1.0,0.0
1,1,1,家庭,1,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
2,1,1,出行,2,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
3,1,2,升级,0,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
4,1,2,房间,1,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0


In [25]:
# 只保留有情感值的词
filt = (result['pos'] == 1) | (result['neg'] == -1)
result = result[filt]
result.drop(columns='emotion',axis=1,inplace=True)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
result['amend_weight'] = result['pos'] + result['neg']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
result

,index_server,index_content,word,index_word,pos,neg,amend_weight
0,1,1,适合,0,1.0,0.0,1.0
5,1,2,延迟,2,0.0,-1.0,-1.0
15,1,3,免费,7,1.0,0.0,1.0
22,1,5,卫生,1,1.0,0.0,1.0
29,1,6,遗憾,5,0.0,-1.0,-1.0
...,...,...,...,...,...,...,...
229904,50,20986,陈旧,4,0.0,-1.0,-1.0
229910,50,20986,剥落,10,0.0,-1.0,-1.0
229918,50,20986,卫生,18,1.0,0.0,1.0
229930,50,20987,喜欢,9,1.0,0.0,1.0


In [28]:
result[result['amend_weight']==1]

,index_server,index_content,word,index_word,pos,neg,amend_weight
0,1,1,适合,0,1.0,0.0,1.0
15,1,3,免费,7,1.0,0.0,1.0
22,1,5,卫生,1,1.0,0.0,1.0
37,1,7,确实,4,1.0,0.0,1.0
42,1,8,整洁,3,1.0,0.0,1.0
...,...,...,...,...,...,...,...
229821,50,20973,气派,13,1.0,0.0,1.0
229861,50,20978,气派,9,1.0,0.0,1.0
229866,50,20979,卫生,0,1.0,0.0,1.0
229918,50,20986,卫生,18,1.0,0.0,1.0


## 分酒店计算每条评论的情感值

In [29]:
# 计算判断离群值阈值
# 输入describe的结果
# 输出上界和下界
def outlier(des):
    # 计算iqr：数据四分之三分位值与四分之一分位值的差
    iqr = des.loc['75%','amend_weight'] - des.loc['25%','amend_weight']
    # 根据iqr计算异常值判断阈值
    u_th = des.loc['75%','amend_weight'] + 1.5*iqr # 上界
    l_th = des.loc['25%','amend_weight'] - 1.5*iqr # 下界
    return [u_th, l_th]

In [30]:
# 情感值大于离群值上界的情感等级都为最高
# 情感值小于离群值下界的情感等级都为最低
def classify(x, bins):
    if x<bins[1]:
        return 1
    elif bins[1]<=x<bins[2]:
        return 2
    elif bins[2]<=x<bins[3]:
        return 3
    elif bins[3]<=x<bins[4]:
        return 4
    elif bins[4]<=x:
        return 5

In [31]:
# 计算判断离群值阈值
# 输入describe的结果
# 输出上界和下界
def outlier(des):
    # 计算iqr：数据四分之三分位值与四分之一分位值的差
    iqr = des.loc['75%','amend_weight'] - des.loc['25%','amend_weight']
    # 根据iqr计算异常值判断阈值
    u_th = des.loc['75%','amend_weight'] + 1.5*iqr # 上界
    l_th = des.loc['25%','amend_weight'] - 1.5*iqr # 下界
    return [u_th, l_th]

In [32]:
for indexServer in range(1,51):
    # 计算每条评论的情感值
    result_ser = result[result['index_server'] == indexServer]
    emotional_value = result_ser.groupby(['index_content'],as_index=False)['amend_weight'].sum()
    
    # 查看离群值
    des = emotional_value.describe()
    plt.figure(1)
    plt.boxplot(emotional_value['amend_weight'])
    plt.savefig(fig_path1 + str(indexServer) + '.png')
    plt.close(1)
    
    # 处理离群值
    u, l = outlier(des)
    filt = (emotional_value['amend_weight'] <= u) & (emotional_value['amend_weight'] >= l)
    not_out = emotional_value[filt]
    
    # 确定划分等级的bins
    plt.figure(2)
    n, bins, patches = plt.hist(not_out['amend_weight'],bins=5)
    plt.xlabel('emotional value')
    plt.ylabel('the number of reviews')
    plt.savefig(fig_path2 + str(indexServer) + '.png')
    plt.close(2)
    
    # 情感分级（1-5，1级最不满意）
    output = reviews[reviews['酒店编号'] == indexServer]
    output['emotional_value'] = (bins[2] + bins[3])/2  # 初始值都设为中立
    output['index'] = output.index.values
    
    # 若index_content =1,则对应第0条评论
    # 将情感值合并到源文件中
    # 情感值被删去的评论默认情感值为0
    step = output.index.values[0]  # 提取dataframe后索引有变化
    for row in range(0, len(emotional_value)):
        IndexContent = emotional_value.iloc[row, 0]
        amend_weight = emotional_value.iloc[row, 1]
        output.iloc[IndexContent - step - 1, 10] = amend_weight
        
    # 分级
    # range为由直方图得到的区间bins,等分成5个等级
    output['情感等级'] = 3
    output['情感等级'] = output.emotional_value.map(lambda x: classify(x, bins))
    
    output.to_csv(result_path_u + str(indexServer) + 'utf-8.csv',encoding='utf-8', index=False)
    output.to_csv(result_path_g + str(indexServer) + 'gbk.csv',encoding='gbk', index=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [33]:
output

,酒店名称,评论日期,评论内容,入住房型,主题,评论对主题贡献度,主题关键字,评论分词,评价方面,酒店编号,emotional_value,index,情感等级
20749,H50,2020-01-01,位置好，服务也不错，推荐！,童趣房,5.0,0.8283,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[位置, 服务, 不错, 推荐]",性价比,50,1.0,20749,4
20750,H50,2020-01-01,经过装修后设施一流。房价偏高,高级大床房,5.0,0.7207,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[装修, 设施, 一流, 房价, 偏, 高]",性价比,50,1.0,20750,4
20751,H50,2020-01-02,设施旧一点，还算干净，性价比高！,高级双床房,5.0,0.4261,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[设施, 旧, 一点, 算, 干净, 性价比, 高]",性价比,50,0.5,20751,3
20752,H50,2020-01-02,出差特别好，价格不贵，周边方便！就是隔音不好,高级双床房,5.0,0.4859,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[出差, 特别, 价格, 贵, 周边, 隔音, 不好]",性价比,50,0.0,20752,2
20753,H50,2020-01-02,没吃早餐，周边去哪都方便,高级双床房,5.0,0.7853,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[吃, 早餐, 周边]",性价比,50,0.5,20753,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20984,H50,2020-08-22,不错，性价比高,高级双床房,5.0,0.7856,"不错, 环境, 服务, 早餐, 位置, 交通, 设施, 房间, 性价比, 高","[不错, 性价比, 高]",性价比,50,0.5,20984,3
20985,H50,2020-08-24,酒店是老字号宾馆，虽然有装修过但设施比较陈旧房间空调比较小，卫生间里的墙壁有剥落现象，浴室的...,高级双床房,2.0,0.5633,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","[老字号, 宾馆, 装修, 设施, 陈旧, 房间, 空调, 卫生间, 里, 墙壁, 剥落, ...",位置,50,-1.0,20985,1
20986,H50,2020-08-24,本人觉得还好，价格偏高，没有很高档，服务是很好，设施偏老，也不会很大，喜欢楼下的早茶。,高级大床房,2.0,0.4539,"房间, 位置, 旧, 地理位置, 住, 设施, 机场, 早餐, 出行, 出差","[还好, 价格, 偏, 高, 高档, 服务, 设施, 偏, 很大, 喜欢, 楼下, 早茶]",位置,50,1.0,20986,4
20987,H50,2020-08-24,隔音极差，睡眠让人崩溃。,高级大床房,6.0,0.5919,"房间, 差, 服务态度, 入住, 疫情, 空调, 太, 期间, 点, 体验","[隔音, 极差, 睡眠, 崩溃]",设施,50,-1.0,20987,1
